### Cleanup for the foursight check
carry this to fs

In [2]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import *
import time
time1 = time.time()

# first used this query on browser to find out problematic cases
# old hic run types are expected to miss that, also file prov and sbg runs

# '/search/?source_experiments=No+value&type=FileProcessed&workflow_run_outputs.uuid%21=No+value'


my_key = get_key('koray_data')

# all cases from bwa-mem
url1 = '/search/?source_experiments=No+value&type=FileProcessed&workflow_run_outputs.workflow.title=Alignment+for+Hi-C+using+bwa-mem+0.2.6'
res1 = ff_utils.search_metadata(url1, my_key)
print(len(res1))


#all cases from step2
url2 = '/search/?source_experiments=No+value&type=FileProcessed&workflow_run_outputs.workflow.title=Hi-C+Post-alignment+Processing+0.2.6'
res2 = ff_utils.search_metadata(url2, my_key)
print(len(res2))

16


In [13]:
# find input files (fastq) and get experiments, patch files
for a_file in res1:
    print(a_file['accession'])
    previous_files = a_file['workflow_run_outputs'][0]['input_files']
    previous_files = [i for i in previous_files if i.get('workflow_argument_name', '') in ['fastq1', 'fastq2']]
    exps = []
    for a_case in previous_files:
        acc = a_case['value']['uuid']
        resp = ff_utils.get_metadata(acc, my_key)
        exps.extend([i['uuid'] for i in resp['experiments']])
    exps = list(set(exps))
    ff_utils.patch_metadata({'source_experiments': exps}, a_file['uuid'], my_key)    

4DNFIAXZLSCT
4DNFIGM4SG56
4DNFIMLJAMFS
4DNFIYJAPGX1
4DNFICEKFQYI
4DNFIDO5OJPX
4DNFI6QKAF7W
4DNFIXTZ71Z9
4DNFIBYMDV8I
4DNFIPLHLXI5
4DNFIR1DYCGW
4DNFI4N94H26
4DNFIQLX8ML3
4DNFIYVTL965
4DNFIXAHPX8O
4DNFIP2SZ4QP


In [16]:
# go through input processed files, and accumulate source_experiments val
for a_file in res2:
    print(a_file['accession'])
    previous_files = a_file['workflow_run_outputs'][0]['input_files']
    previous_files = [i for i in previous_files if i.get('workflow_argument_name', '') in ['input_bams']]
    exps = []
    for a_case in previous_files:
        acc = a_case['value']['uuid']
        resp = ff_utils.get_metadata(acc, my_key)
        assert resp['source_experiments']
        exps.extend(resp['source_experiments'])
    exps = list(set(exps))
    print(exps)
    ff_utils.patch_metadata({'source_experiments': exps}, a_file['uuid'], my_key) 

4DNFI8MJRZJ4
['363b5698-108e-40dc-a1ea-9c02a8c5e956']
4DNFIT2X1PMM
['363b5698-108e-40dc-a1ea-9c02a8c5e956']
4DNFIHDYKTU3
['6137fdc2-0f73-47f7-a46f-666efbcee23b']
4DNFID1SPU94
['6137fdc2-0f73-47f7-a46f-666efbcee23b']


In [23]:
# realized that step3 did not add the missing exp from previous step
# find the output files
for clue in res2:
    print(clue['accession'])
    next_files = [a['value']['accession'] for i in clue['workflow_run_inputs'] for a in i['output_files'] if a.get('value')]
    print(next_files)
    try:
        assert len(next_files) == 3
    except:
        continue
    for a_new_file in next_files:
        a_file = ff_utils.get_metadata(a_new_file, my_key)

        print(a_file['accession'], 'in progress')
        previous_files = a_file['workflow_run_outputs'][0]['input_files']
        previous_files = [i for i in previous_files if i.get('workflow_argument_name', '') in ['input_pairs']]
        exps = []
        for a_case in previous_files:
            acc = a_case['value']['uuid']
            resp = ff_utils.get_metadata(acc, my_key)
            assert resp['source_experiments']
            exps.extend(resp['source_experiments'])
        exps = list(set(exps))
        print(exps)
        ff_utils.patch_metadata({'source_experiments': exps}, a_file['uuid'], my_key) 

    

4DNFI8MJRZJ4
['4DNFI67T52AI', '4DNFIF9BDCNI', '4DNFIIDNWL5I']
4DNFI67T52AI in progress
['c9cce412-b4a6-4ba6-9fd9-a83f3df032e9', '363b5698-108e-40dc-a1ea-9c02a8c5e956']
4DNFIF9BDCNI in progress
['c9cce412-b4a6-4ba6-9fd9-a83f3df032e9', '363b5698-108e-40dc-a1ea-9c02a8c5e956']
4DNFIIDNWL5I in progress
['c9cce412-b4a6-4ba6-9fd9-a83f3df032e9', '363b5698-108e-40dc-a1ea-9c02a8c5e956']
4DNFIT2X1PMM
[]
4DNFIHDYKTU3
['4DNFI5VLBU41', '4DNFIUATRW3Z', '4DNFIOF2U3HX']
4DNFI5VLBU41 in progress
['6137fdc2-0f73-47f7-a46f-666efbcee23b', 'bfba3c59-d107-42cc-aa28-bb592ffd815e']
4DNFIUATRW3Z in progress
['6137fdc2-0f73-47f7-a46f-666efbcee23b', 'bfba3c59-d107-42cc-aa28-bb592ffd815e']
4DNFIOF2U3HX in progress
['6137fdc2-0f73-47f7-a46f-666efbcee23b', 'bfba3c59-d107-42cc-aa28-bb592ffd815e']
4DNFID1SPU94
[]
